In [1]:
# import SAGA code

import SAGA
from SAGA import ObjectCuts as C
from SAGA.utils import *

print('SAGA code version', SAGA.__version__)

SAGA code version 0.3.4


In [2]:
# import other modules

from collections import defaultdict
from astropy.table import Table
from astropy.coordinates import SkyCoord
from easyquery import Query

In [3]:
# initialize SAGA objects (Database, HostCatalog, ObjectCatalog)
# NOTE: change the path of `root_dir` to your SAGA dropbox path

saga_database = SAGA.Database(root_dir='/home/yymao/Dropbox/Academia/Collaborations/SAGA')

saga_database_new_dr12 = SAGA.Database()
saga_database_new_dr12.base_file_path_pattern = '/home/yymao/Documents/Research/SAGA/new_base_catalogs/base_dr12_nsa{}.fits.gz'

saga_database_new_dr14 = SAGA.Database()
saga_database_new_dr14.base_file_path_pattern = '/home/yymao/Documents/Research/SAGA/new_base_catalogs/base_dr14_nsa{}.fits.gz'

saga_object_catalog = SAGA.ObjectCatalog(saga_database)
saga_object_catalog_new_dr12 = SAGA.ObjectCatalog(saga_database_new_dr12)
saga_object_catalog_new_dr14 = SAGA.ObjectCatalog(saga_database_new_dr14)

saga_host_catalog = SAGA.HostCatalog(saga_database)

In [4]:
def get_counts(base_list):
    data = defaultdict(list)

    for t in base_list:
        host_id = t['HOST_NSAID'][0]

        data['SAGA name'].append(saga_host_catalog.id_to_name(host_id))
        data['NSAID'].append(host_id)
        data['sat'].append((C.is_sat & C.is_clean & C.has_spec).count(t))
        data['low_z'].append((C.is_low_z & C.is_clean & C.has_spec).count(t))

        for i in (-1, 0, 1, 2, 3, 4):
            data['zq_{}'.format(i)].append(Query('ZQUALITY=={}'.format(i)).count(t))

        for i in (-1, 0, 1, 2, 3, 4, 5):
            data['rm_{}'.format(i)].append(Query('REMOVE=={}'.format(i)).count(t))

    return Table(data)

## load base catalogs

In [5]:
# specify columns to load
columns = 'OBJID HOST_NSAID RA DEC REMOVE SATS SPEC_Z ZQUALITY TELNAME SPEC_REPEAT r_mag coord'.split()
columns.extend(get_sdss_colors()[:3])
columns.extend(map('{}_err'.format, get_sdss_colors()[:3]))

cuts = C.is_galaxy & C.fibermag_r_cut

# load base catalogs (will take a while)
paper1 = saga_object_catalog.load(hosts='paper1', cuts=cuts, columns=columns, return_as='list')
paper1_new_dr12 = saga_object_catalog_new_dr12.load(hosts='paper1', cuts=cuts, columns=columns, return_as='list')
paper1_new_dr14 = saga_object_catalog_new_dr14.load(hosts='paper1', cuts=cuts, columns=columns, return_as='list')

## tables of counts

In [6]:
get_counts(paper1)

SAGA name,NSAID,sat,low_z,zq_-1,zq_0,zq_1,zq_2,zq_3,zq_4,rm_-1,rm_0,rm_1,rm_2,rm_3,rm_4,rm_5
str11,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
Gilgamesh,166313,2,9,33770,0,109,37,474,2910,34426,0,11,154,1687,116,906
Odyssey,147100,9,14,33868,0,117,53,307,2509,33723,0,4,153,2105,145,724
Dune,165536,1,8,32808,0,171,128,388,3775,34734,0,17,155,1389,141,834
AnaK,61945,3,7,35070,0,166,41,573,2631,35281,0,18,235,1963,139,845
Narnia,132339,2,5,36935,0,146,20,212,1959,35998,0,6,203,2150,104,811
OBrother,149781,4,4,33147,0,91,41,162,1304,31563,0,6,98,2062,47,969
StarTrek,33446,2,5,33578,0,102,8,124,1639,32015,0,9,55,2478,102,792
Catch22,150887,6,11,31694,0,254,144,156,1392,30641,0,19,80,1596,84,1220
ScoobyDoo,161174,4,8,34573,0,37,2,43,1000,33441,0,2,71,1316,63,762


In [7]:
get_counts(paper1_new_dr12)

SAGA name,NSAID,sat,low_z,zq_-1,zq_0,zq_1,zq_2,zq_3,zq_4,rm_-1,rm_0,rm_1,rm_2,rm_3,rm_4,rm_5
str11,int32,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
Gilgamesh,166313,2,11,33777,0,111,36,474,2902,37267,0,0,0,22,3,8
Odyssey,147100,10,16,33856,0,119,54,307,2519,36814,0,0,2,26,4,9
Dune,165536,1,9,32804,0,175,129,387,3775,37240,0,1,0,22,4,3
AnaK,61945,8,14,35062,0,169,41,579,2630,38445,0,0,2,26,6,2
Narnia,132339,2,5,36935,0,146,21,214,1956,39230,0,0,0,29,2,11
OBrother,149781,4,5,33153,0,91,41,162,1298,34718,0,0,0,18,0,9
StarTrek,33446,3,6,33581,0,102,8,123,1637,35426,0,0,0,16,4,5
Catch22,150887,6,12,31697,0,258,144,156,1385,33614,0,0,1,10,4,11
ScoobyDoo,161174,4,8,34577,0,36,2,43,997,35635,0,0,1,9,4,6


In [8]:
get_counts(paper1_new_dr14)

SAGA name,NSAID,sat,low_z,zq_-1,zq_0,zq_1,zq_2,zq_3,zq_4,rm_-1,rm_0,rm_1,rm_2,rm_3,rm_4,rm_5
str11,int32,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
Gilgamesh,166313,2,9,33739,0,105,33,464,2861,33960,0,21,145,1663,511,902
Odyssey,147100,9,14,33834,0,120,51,297,2496,33236,0,8,150,2073,608,723
Dune,165536,1,7,32670,0,162,118,384,3813,34281,0,42,130,1365,500,829
AnaK,61945,3,6,34650,0,259,40,574,2769,34006,0,49,207,1937,1254,839
Narnia,132339,2,4,37051,0,167,19,202,1784,35209,0,35,175,2145,850,809
OBrother,149781,4,4,33147,0,78,37,157,1277,31276,0,7,97,2050,303,963
StarTrek,33446,2,4,33586,0,102,8,119,1616,31399,0,31,37,2472,701,791
Catch22,150887,6,10,31732,0,240,142,152,1372,30187,0,43,62,1592,533,1221
ScoobyDoo,161174,4,7,34836,0,29,2,42,976,33189,0,19,55,1335,517,770


## check satellites

In [9]:
host_idx = 3
is_sat = (C.is_sat & C.is_clean & C.has_spec)

In [10]:
is_sat.filter(paper1[host_idx])

OBJID,HOST_NSAID,RA,DEC,REMOVE,SATS,SPEC_Z,ZQUALITY,TELNAME,SPEC_REPEAT,r_mag,coord,ug,gr,ri,ug_err,gr_err,ri_err
,,,,,,,,,,,"deg,deg",,,,,,
int64,int64,float64,float64,int64,int64,float32,int64,str6,str48,float32,object,float32,float32,float32,float32,float32,float32
1237666408439939282,61945,354.350599991,0.390802769345,-1,1,0.00887051,4,NSA,SDSS+NSA,13.6617,"354.35059999083205,0.3908027693448825",0.870279,0.405699,0.239671,0.0153288,0.00391624,0.00231464
1237663277925204111,61945,354.195229736,0.62342377148,-1,1,0.00880544,4,NSA,SDSS+NSA,15.6154,"354.1952297364415,0.6234237714801917",1.00401,0.414283,0.206417,0.0143609,0.00361622,0.00430413
1237666408439677694,61945,353.778805283,0.301058523211,-1,1,0.008,3,MMT,MMT,21.2784,"353.77880528268133,0.30105852321103627",0.476309,1.38735,0.58955,0.81189,0.209817,0.113035


In [11]:
is_sat.filter(paper1_new_dr12[host_idx])

OBJID,HOST_NSAID,RA,DEC,REMOVE,SATS,SPEC_Z,ZQUALITY,TELNAME,SPEC_REPEAT,r_mag,coord,ug,gr,ri,ug_err,gr_err,ri_err
,,,,,,,,,,,"deg,deg",,,,,,
int64,int32,float64,float64,int16,int16,float32,int16,str6,str48,float32,object,float32,float32,float32,float32,float32,float32
1237663277925204111,61945,354.195229736,0.62342377148,-1,1,0.008789,4,MMT,SDSS+MMT+AAT+NSA,15.6154,"354.1952297364415,0.6234237714801917",1.00401,0.414283,0.206417,0.0143609,0.00361622,0.00430413
1237666408439873612,61945,354.116125091,0.287918010119,-1,1,0.008995,4,AAT,AAT,20.6799,"354.1161250910807,0.2879180101190925",-0.236053,0.22813,-0.600939,0.101184,0.0578393,0.104102
1237666408439873555,61945,354.128282396,0.294928659376,-1,1,0.00889738,4,SDSS,SDSS,24.7077,"354.12828239564715,0.29492865937609997",0.657375,-6.79342,0.417427,0.0489569,4.74704,6.36414
1237666408439939273,61945,354.359231933,0.387757626686,-1,1,0.009115,4,MMT,SDSS+MMT+AAT,18.4048,"354.3592319332698,0.38775762668616",0.30201,-0.0517807,-0.5721,0.0505896,0.023622,0.0499384
1237666408439677694,61945,353.778805283,0.301058523211,-1,1,0.008,3,MMT,MMT,21.2784,"353.77880528268133,0.30105852321103627",0.476309,1.38735,0.58955,0.81189,0.209817,0.113035
1237666408439939302,61945,354.353927451,0.383455165908,-1,1,0.00911429,4,SDSS,SDSS,22.1533,"354.3539274510332,0.3834551659076629",-0.204454,-0.0589371,0.333857,0.292009,0.206224,0.252724
1237666408439939279,61945,354.340218967,0.39146506493,-1,1,0.00884914,4,SDSS,SDSS+AAT,17.3618,"354.34021896734987,0.39146506493004757",0.420967,0.0300541,-0.0157146,0.0698504,0.028994,0.0401843
1237666408439939282,61945,354.350599991,0.390802769345,-1,1,0.008886,4,MMT,SDSS+MMT+NSA,13.6617,"354.35059999083205,0.3908027693448825",0.870279,0.4057,0.23967,0.0153288,0.00391624,0.00231464


In [12]:
is_sat.filter(paper1_new_dr14[host_idx])

OBJID,HOST_NSAID,RA,DEC,REMOVE,SATS,SPEC_Z,ZQUALITY,TELNAME,SPEC_REPEAT,r_mag,coord,ug,gr,ri,ug_err,gr_err,ri_err
,,,,,,,,,,,"deg,deg",,,,,,
int64,int32,float64,float64,int16,int16,float32,int16,str6,str48,float32,object,float32,float32,float32,float32,float32,float32
1237666408439677694,61945,353.778805282,0.301058517059,-1,1,0.008,3,MMT,MMT,21.3007,"353.7788052821856,0.3010585170592043",0.478346,1.38623,0.589331,0.812941,0.209759,0.113029
1237666408439939282,61945,354.350599991,0.390802769345,-1,1,0.008886,4,MMT,MMT+NSA+SDSS,13.6796,"354.35059999083205,0.3908027693448825",0.886654,0.406622,0.242669,0.0153288,0.00391624,0.00231464
1237663277925204111,61945,354.195229736,0.62342377148,-1,1,0.008789,4,MMT,MMT+AAT+SDSS+NSA,15.6313,"354.1952297364415,0.6234237714801917",1.01855,0.415103,0.209079,0.0143609,0.00361622,0.00430413
